# Export data to files with Suitcase

In this notebook you will:

* Export saved data to CSV and TIFF.
* See a preview of how exporting data live during a scan works.

## Configuration
Below, we will connect to EPICS IOC(s) controlling simulated hardware in lieu of actual motors, detectors. The IOCs should already be running in the background. Run this command to verify that they are running: it should produce output with RUNNING on each line. In the event of a problem, edit this command to replace `status` with `restart all` and run again.

In [1]:
!supervisorctl -c supervisor/supervisord.conf status

decay                            RUNNING   pid 4847, uptime 0:00:34
mini_beamline                    RUNNING   pid 4848, uptime 0:00:34
random_walk                      RUNNING   pid 4849, uptime 0:00:34
random_walk_horiz                RUNNING   pid 4850, uptime 0:00:34
random_walk_vert                 RUNNING   pid 4851, uptime 0:00:34
simple                           RUNNING   pid 4852, uptime 0:00:34
thermo_sim                       RUNNING   pid 4853, uptime 0:00:34
trigger_with_pc                  RUNNING   pid 4854, uptime 0:00:34


In [2]:
%run scripts/beamline_configuration.py

## Scalar data to CSV

In [3]:
RE(scan([ph], motor_ph, -1, 1, 5), sample_comp='Ni', barcode='12345')



Transient Scan ID: 3     Time: 2020-06-13 20:06:02
Persistent Unique Scan ID: '3a6c339e-9349-406b-9f1c-060a1eacc7d8'
New stream: 'primary'


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

+-----------+------------+------------+------------+
|   seq_num |       time |   motor_ph |     ph_det |
+-----------+------------+------------+------------+
|         1 | 20:06:02.3 |     -1.000 |     100031 |
|         2 | 20:06:02.3 |     -0.500 |     101100 |
|         3 | 20:06:02.4 |      0.000 |     101105 |
|         4 | 20:06:02.4 |      0.500 |     100706 |
|         5 | 20:06:02.4 |      1.000 |      98442 |


+-----------+------------+------------+------------+
generator scan ['3a6c339e'] (scan num: 3)





('3a6c339e-9349-406b-9f1c-060a1eacc7d8',)

In [4]:
import suitcase.csv

In [5]:
suitcase.csv.export(db[-1].documents(fill=True), 'my_exported_files/')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/3a6c339e-9349-406b-9f1c-060a1eacc7d8-primary.csv')]}

In [6]:
suitcase.csv.export(db[-1].documents(fill=True), 'my_exported_files/', file_prefix='{start[sample_comp]}-{start[barcode]}-')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni-12345-primary.csv')]}

In [7]:
suitcase.csv.export(db[-1].documents(fill=True), 'my_exported_files/', file_prefix='{start[sample_comp]}/{start[barcode]}-')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345-primary.csv')]}

## Image data to TIFF

In [8]:
RE(scan([spot], mtr_spotx, -1, 1, 5), sample_comp='Ni', barcode='12345')



Transient Scan ID: 4     Time: 2020-06-13 20:06:02
Persistent Unique Scan ID: '73ae3701-3f2f-4b5a-b7cc-86a89806a72b'


New stream: 'primary'


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

+-----------+------------+-------------+------------+
|   seq_num |       time | motor_spotx |   spot_roi |
+-----------+------------+-------------+------------+
|         1 | 20:06:02.9 |          -1 | 1079623091 |


|         2 | 20:06:03.1 |          -0 | 1073401395 |


|         3 | 20:06:03.1 |           0 | 1063914616 |


|         4 | 20:06:03.3 |           0 | 1059088944 |


|         5 | 20:06:03.3 |           1 | 1056186945 |
+-----------+------------+-------------+------------+
generator scan ['73ae3701'] (scan num: 4)





('73ae3701-3f2f-4b5a-b7cc-86a89806a72b',)

### TIFF stack, a 3D block of images in one file

In [9]:
import suitcase.tiff_stack

In [10]:
suitcase.tiff_stack.export(db[-1].documents(fill=True), 'my_exported_files/', file_prefix='{start[sample_comp]}/{start[barcode]}-')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345-primary-spot_img.tiff')]}

### TIFF series

In [11]:
import suitcase.tiff_series

Unlike CSV or TIFF stack, this exporter creates separate file per row (Event document). We can include information from a specific Event in the filename. Here, we include the motor position.

In [12]:
suitcase.tiff_series.export(db[-1].documents(fill=True), 'my_exported_files/', file_prefix='{start[sample_comp]}/{start[barcode]}/{event[data][motor_spotx]}-')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/-1.0-primary-spot_img-0.tiff'),
  PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/-0.5-primary-spot_img-1.tiff'),
  PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/0.0-primary-spot_img-2.tiff'),
  PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/0.5-primary-spot_img-3.tiff'),
  PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/1.0-primary-spot_img-4.tiff')]}

## "Beta" feature: Set up live exporting

*We plan to streamline this in the future, but here is a demonstration that it is possible.*

In [13]:
# Execute this once to configure RE to send data to an exporter.
# At the beamline, this would normally be done in a script automatically at startup.

from event_model import RunRouter
from databroker.core import discover_handlers

def factory(name, start_doc):

    serializer = suitcase.tiff_series.Serializer(
        'my_exported_files/', file_prefix='{start[sample_comp]}/{start[barcode]}-{start[uid]:.8}-')

    def cb(name, doc):
        serializer(name, doc)

    return [cb], []


rr = RunRouter([factory], discover_handlers())
RE.subscribe(rr)

2

In [14]:
RE(count([spot], num=30, delay=1), sample_comp='Lab6', barcode='12345')



Transient Scan ID: 5     Time: 2020-06-13 20:06:03
Persistent Unique Scan ID: '7dc4d032-c804-4a1c-bcc6-25c8f7e0a10c'
New stream: 'primary'


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

+-----------+------------+------------+
|   seq_num |       time |   spot_roi |
+-----------+------------+------------+
|         1 | 20:06:03.8 | 1054422090 |


|         2 | 20:06:04.8 | 1103010954 |


|         3 | 20:06:05.8 | 1130253118 |


...

|        28 | 20:06:30.8 | 1084048839 |


|        29 | 20:06:31.8 | 1055501250 |


|        30 | 20:06:32.8 | 1105855314 |


+-----------+------------+------------+
generator count ['7dc4d032'] (scan num: 5)





('7dc4d032-c804-4a1c-bcc6-25c8f7e0a10c',)